In [4]:
import pandas as pd 

indonesia_full = pd.read_csv('https://raw.githubusercontent.com/PeishanLi/G5055_Practicum_Project2/main/Data/Indonesia%20Data%20Without%20Disaggregation.csv')

In [8]:
# Indonesia Data including '[Units]' Data 
indonesia_full['[Units]'].value_counts()

PERCENT                   638
NUMBER                    354
PER_100000_POP             69
INDEX                      65
CU_USD                     50
KMSQ                       47
CON_USD                    33
NUM_TH                     30
Ratio                      29
CU_USD_M                   24
PER_1000_LIVE_BIRTHS       24
CON_USD_M                  20
HA_TH                      18
W_PER_CAPITA               16
NUM_M                       8
SCORE                       8
CUR_LCU_M                   8
PER_100_POP                 8
PER_1000_POP                8
TEU                         8
KG_PER_CON_USD              7
TONNES_M                    7
USD/m3                      7
MJ_PER_GDP_CON_PPP_USD      7
PER_POP_U5                  6
BOOL                        6
T_PER_HA                    6
TONNES                      6
mgr/m^3                     5
PER_1000000_POP             3
Name: [Units], dtype: int64

In [14]:
series_units = indonesia_full[['SeriesCode','[Units]']]
series_units = series_units.drop_duplicates()
series_units

,SeriesCode,[Units]
0,SI_POV_DAY1,PERCENT
8,SI_POV_NAHC,PERCENT
11,SI_COV_POOR,PERCENT
12,SI_COV_CHLD,PERCENT
13,SI_COV_UEMP,PERCENT
...,...,...
1484,BX_TRF_PWKR,PERCENT
1493,GF_FRN_FDI,CU_USD_M
1501,DT_TDS_DECT,PERCENT
1509,IT_USE_ii99,PER_100_POP


In [ ]:
import os
import glob

path = '../../Data/'
extension = 'csv'
os.chdir(path)
result = glob.glob('*.{}'.format(extension))
print(result)


In [ ]:
for file in 